# creat model

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

creat embedding layer

In [3]:
def make_embedding(): 
    inp = Input(shape=(100,100,3), name='input_image')
    
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [4]:
embedding = make_embedding()

In [5]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [6]:
def make_siamese_model(): 
    input_image = Input(name='input_img', shape=(100,100,3))
    validation_image = Input(name='validation_img', shape=(100,100,3))
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [7]:
siamese_model = make_siamese_model()
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_img (InputLayer)    [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_img[0][0]',              
                                                                  'validation_img[0][

# load and prepeocess data

In [8]:
import cv2 
import numpy as np
import random 
import matplotlib.pyplot as plt 
import os 

In [9]:
%cd '/content/drive/MyDrive/faceR'

/content/drive/MyDrive/faceR


In [10]:
Neg_path=os.path.join('data','negative')
Pos_path=os.path.join('data','positive')
Anc_path=os.path.join('data','anchor')

In [ ]:
os.makedirs(Neg_path)
os.makedirs(Pos_path)
os.makedirs(Anc_path)

In [ ]:
for directory in os.listdir('lfw'):
  for files in os.listdir(os.path.join('lfw',directory)):
    Ex_path=os.path.join('lfw',directory,files)
    New_path= os.path.join(Neg_path, files)
    os.replace(Ex_path,New_path)

In [ ]:
import uuid

In [ ]:
cam=cv2.VideoCapture(0)
while True:
    _,frame=cam.read()
    frame=cv2.flip(frame,1)
    frame = frame[120:120+250,200:200+250, :]
    if cv2.waitKey(1)& 0XFF ==ord('p'):
        image_name=os.path.join(Pos_path,'{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(image_name,frame)
    
    if cv2.waitKey(1)& 0XFF ==ord('a'):
        image_name=os.path.join(Anc_path,'{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(image_name,frame)
    
    cv2.imshow('fram',frame)
    if cv2.waitKey(1)& 0XFF ==27:
        break

creat helper fuction 

In [11]:
def normalize(image):
  image=(image/127.5)-1
  return image
def resize(image,height,width):
  image=tf.image.resize(image,(height,width),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  return image

def load_image(image_path):
  image=tf.io.read_file(image_path)
  image=tf.image.decode_jpeg(image,3)
  image=tf.cast(image,tf.float32)
  image=resize(image,100,100)
  image=normalize(image)
  return image 


In [12]:
def load_anchor_positive_with_label(image_path,positive_path):
  anchor=load_image(image_path)
  positive=load_image(positive_path)
  label=tf.ones(1)
  return anchor , positive , label

def load_anchor_negative_with_label(image_path,negative_path):
  anchor=load_image(image_path)
  negative=load_image(negative_path)
  label=tf.zeros(1)
  return anchor , negative , label

In [13]:
anchor=tf.data.Dataset.list_files(Anc_path+'/*.jpg').take(300)
positive=tf.data.Dataset.list_files(Pos_path+'/*.jpg').take(300)
negative=tf.data.Dataset.list_files(Neg_path+'/*.jpg').take(300)

In [14]:
anc_pos=tf.data.Dataset.zip((anchor,positive))
anc_neg=tf.data.Dataset.zip((anchor,negative))

In [15]:
anc_pos_data=anc_pos.map(load_anchor_positive_with_label)
anc_neg_data=anc_neg.map(load_anchor_negative_with_label)

In [16]:
data = anc_pos_data.concatenate(anc_neg_data)

In [17]:
data=data.shuffle(1000)
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)

In [18]:
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)

## train model 

In [19]:
from tensorflow.keras.metrics import Recall,Precision

In [20]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(1e-4) 

checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [21]:
@tf.function
def train_Step(batch):
    with tf.GradientTape() as tape:    
        X = batch[:2]
        y = batch[2]
        yhat = siamese_model(X, training=True)
        loss = binary_cross_loss(y, yhat)
    print(loss)    
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    return loss

In [22]:
def Train(data, EPOCHS):
    
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        r = Recall()
        p = Precision()
  
        for idx, batch in enumerate(data):
            loss = train_Step(batch)
            yhat = siamese_model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print("train_loss",loss.numpy(),"Recall",r.result().numpy(),'Precision', p.result().numpy())
        
     
        if epoch % 5 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

In [23]:
EPOCHS =10
Train(train_data, EPOCHS)


 Epoch 1/10
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
27/27 [==============================] - 71s 353ms/step
train_loss 0.0006839564 Recall 0.9766355 Precision 0.88559324

 Epoch 2/10
27/27 [==============================] - 70s 311ms/step
train_loss 0.000536924 Recall 0.99512196 Precision 0.96682465

 Epoch 3/10
27/27 [==============================] - 68s 311ms/step
train_loss 0.00043502217 Recall 0.99033815 Precision 0.9855769

 Epoch 4/10
27/27 [==============================] - 69s 310ms/step
train_loss 0.0012850091 Recall 0.99523807 Precision 0.9905213

 Epoch 5/10
27/27 [==============================] - 63s 321ms/step
train_loss 6.7719564e-05 Recall 1.0 Precision 0.9954338

 Epoch 6/10
27/27 [==============================] - 67s 321ms/step
train_loss 0.0024731914 Recall 1.0 Precision 0.9855769

 Epoch 7/10
27/27 [==============================] - 61s 313ms/step
tra

In [24]:
recall=tf.keras.metrics.Recall()
precision=tf.keras.metrics.Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_model.predict([test_input, test_val])
    recall.update_state(y_true, yhat)
    precision.update_state(y_true,yhat) 

print(recall.result().numpy(), precision.result().numpy())

0.9892473 0.9892473


In [25]:
siamese_model.save('my_model.h5')

test Real time 

In [26]:
model = tf.keras.models.load_model('my_model.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})